In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
from keras.applications.densenet import DenseNet121
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
'''
In this section the CIFAR10 dataset is loaded and displayed
the class variables are also defined
The data is then scaled and transformed into one-hotencoding
'''
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
          'dog', 'frog', 'horse', 'ship', 'truck']
W_grid, L_grid = 10, 10
fig, axes = plt.subplots(L_grid, W_grid, figsize=(17, 17))
axes = axes.ravel()
n_train = len(X_train)

for i in np.arange(0, W_grid * L_grid):
    index = np.random.randint(0, n_train)
    axes[i].imshow(X_train[index])
    label_index = int(y_train[index])
    axes[i].set_title(labels[label_index], fontsize=8)
    axes[i].axis('off')
plt.subplots_adjust(hspace=0.4)

# data prep
X_train = X_train / 255.0
X_test = X_test / 255.0
y_cat_train = to_categorical(y_train, 10)
y_cat_test = to_categorical(y_test, 10)

In [ ]:
'''
This is where the data is augmented using ImageDataGenerator
'''
batch_size = 32
data_generator = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
train_generator = data_generator.flow(X_train, y_cat_train, batch_size)
steps_per_epoch = X_train.shape[0] // batch_size

In [ ]:
'''
This is where the DenseNet121 model architecture is defined
It uses the Adam optimizer like the basic CNN model
'''
model = Sequential()
base_model = DenseNet121(input_shape=(32, 32, 3), include_top=False, weights='imagenet', pooling='avg')
model.add(base_model)
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

r = model.fit(
    train_generator, 
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_test, y_cat_test)
)

In [ ]:
'''
The model weights are outputted to be used for testing and image classificatoion tasks
'''
model.save('cnn_50_epochs.h5')